In [22]:
import matplotlib.pyplot as plt  # Used for plotting
%matplotlib inline
import pandas as pd  # Home of the DataFrame construct, _the_ most important object for Data Science
import numpy as np
import sys  # Python system library needed to load custom functions
import os
import shutil
from matplotlib.patches import Rectangle  # Allows drawing the bounding boxes of the worm sections
from PIL import Image  # For loading image files
from tqdm import tqdm  # for timing a for loop
import glob
import PIL
from PIL import Image
from tkinter import image_names
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import time
import matplotlib
from gc import set_debug
from pprint import pprint
from re import S
import struct
from turtle import color
from cv2 import mean
from scipy.stats import cauchy, logistic
from PIL import ImageMath, ImageColor, ImageEnhance, ImageFilter
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
!pip install imutils
!pip install docutils

In [5]:
sys.path.append('/home/sagemaker-user/gdsc5-tutorials-public/src')

In [6]:
from Dataset import OnchoDataset
from detection_util import create_predictions
from gdsc_score import get_leaderboard_score
from gdsc_util import download_directory, download_file, load_sections_df, set_up_logging, PROJECT_DIR
from PredictionEvaluator import PredictionEvaluator
import random

set_up_logging()  # Sets up logging to console and the .log file
data_folder = str(PROJECT_DIR / 'data')

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [7]:
section_df = load_sections_df(f'{data_folder}/gdsc_train.csv')
len(section_df)

65687

In [8]:
def getAllFiles(path,extension='jpg'):
    import glob, os
    os.chdir(path)
    return [file for file in glob.glob('*.'+extension)]

In [9]:
def get_file_size(path):
    size = os.path.getsize(path)
    if size < 1024:
        return f"{size} bytes"
    elif size < 1024*1024:
        return f"{round(size/1024, 2)} KB"
    elif size < 1024*1024*1024:
        return f"{round(size/(1024*1024), 2)} MB"
    elif size < 1024*1024*1024*1024:
        return f"{round(size/(1024*1024*1024), 2)} GB"

In [10]:
def get_folder_size(path):
    size = 0
    for path, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(path, f)
            size += os.stat(fp).st_size
   
    if size < 1024:
        return f"{size} bytes"
    elif size < 1024*1024:
        return f"{round(size/1024, 2)} KB"
    elif size < 1024*1024*1024:
        return f"{round(size/(1024*1024), 2)} MB"
    elif size < 1024*1024*1024*1024:
        return f"{round(size/(1024*1024*1024), 2)} GB"

In [11]:
def getLatestFile(path):
    import glob
    import datetime
    import time
    now = datetime.datetime.now()
    current_time = now.strftime('%Y-%m-%d  %H:%M:%S')
    
    if path[-1] is '/':
        folder_path=path+'*'
    else:
        folder_path=path+'/*'        
    list_of_files = glob.glob(folder_path) # * means all if need specific format then *.csv
    t=os.path.getctime
    latest_file = max(list_of_files, key=t)
    # print(time_created)
    print('Latest Modified file: ')
    print(f'Path: {latest_file}')
    print(f'Size: {get_file_size(latest_file)}')
    if path[-1] is '/':
        file=latest_file[len(path):]
    else:
        file=latest_file[len(path)+1:]
    file_date = time.ctime(os.path.getmtime(latest_file))
    file_date = datetime.datetime.strptime(file_date, "%a %b %d %H:%M:%S %Y")
    print("Last modif: %s" % file_date.strftime('%Y-%m-%d  %H:%M:%S'))
    print("Current Time: ", current_time)
    return file

In [12]:
def getLatestFolder(path):
    import glob
    import datetime
    import time
    now = datetime.datetime.now()
    current_time = now.strftime('%Y-%m-%d  %H:%M:%S')
    t=os.path.getctime  
    latest_folder =max(glob.glob(os.path.join(path, '*/')), key=t)
    print('Latest Modified folder: ')
    print(f'Path: {latest_folder}')
    print(f'Size: {get_folder_size(latest_folder)}')
    if path[-1] is '/':
        folder=latest_folder[len(path):-1]
    else:
        folder=latest_folder[len(path)+1:-1]
    file_date = time.ctime(os.path.getmtime(latest_folder))
    file_date = datetime.datetime.strptime(file_date, "%a %b %d %H:%M:%S %Y")
    print("Last modif: %s" % file_date.strftime('%Y-%m-%d  %H:%M:%S'))
    print("Current Time: ", current_time)
    return folder

In [13]:
def countFilesInPath(path):
    import os
    _, _, files = next(os.walk(path))
    print (f'Path: {path}')
    print (f'Number of files: {len(files)}')
    return

In [14]:
def emptyFolder(parent_dir,folder_name,root_path='/home/sagemaker-user/'):  
    import shutil
    folder =root_path+ parent_dir+ folder_name
    print(folder)
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    
    print(f'Folder {folder} is empty now!')
    return

In [15]:
def create_folder(parent_dir,folder_name):
    path = os.path.join(parent_dir, folder_name)
    if os.path.isdir(path):
        print(f"Directory already exits: {path}")
        return
    os.mkdir(path)
    print(f"Directory created: {path}")

In [16]:
def find_largest_smallest_files(folder_path,file_extension='*'):   
    list_of_files = list(filter(os.path.isfile,glob.glob(folder_path + '*.'+extension) ))
    if len(list_of_files)>0:
        max_file = max( list_of_files,key =  lambda x: os.stat(x).st_size)
        print('Max File: ', max_file)
        print(f"Max File size: {get_file_size(max_file)}")
        min_file=min( list_of_files,key =  lambda x: os.stat(x).st_size)
        print('Min File: ', min_file)
        print(f"Min File size: {get_file_size(min_file)}")
    else:
        print(f"no file with {extension} extension found!")  
    return 

In [26]:
def current_time():
    return time.strftime("%Y-%m-%d--%H-%M-%S")

In [36]:
def create_zip_file_from_folder(folder_path,storage_path,zip_file_name='zipped'):
    now=current_time()
    file_name=zip_file_name+'-'+now
    shutil.make_archive(file_name, 'zip', folder_path)
    if storage_path != os.getcwd()+'/':
        shutil.move(file_name+'.zip', storage_path)
    print(f"{file_name+'.zip'} saved to {storage_path}")
    return

In [38]:
root_path='/home/sagemaker-user/'
train_label_path=root_path+'river-blindness/content/new/labels/train/'
valid_label_path=root_path+'river-blindness/content/new/labels/valid/'

create_zip_file_from_folder(train_label_path,root_path+'river-blindness/changes/')

zipped-2022-08-03--21-44-56.zip saved to /home/sagemaker-user/river-blindness/changes/
